In [33]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

In [73]:
train_data = pd.read_csv('../data_new/train.csv')
test = pd.read_csv('../data_new/test.csv')

C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\2619112913.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data_new/train.csv')


In [54]:
target = train_data.iloc[:,-29:]
target_cols = train_data.iloc[:,-29:].columns

In [55]:
train_data.Blood_Month_sample = train_data.Blood_Month_sample.fillna(train_data.Blood_Month_sample.mean())
train_data.Blood_Month_sample = train_data.Blood_Month_sample.astype('int64')
# train_data.General_cofactors = train_data.General_cofactors.fillna(train_data.General_cofactors.mean())
# train_data.General_cofactors = train_data.General_cofactors.astype('int64')

In [ ]:
# train_data.General_ocofactors = train_data.General_cofactors.fillna(train_data.General_cofactors.mean())
# train_data.General_cfactors = train_data.General_cofactors.astype('int64')
# train_data.General_cofactors.value_counts()

In [56]:
train = train_data.drop(target, axis=1)

In [57]:
def multival_onehot_enc(df, col_names):
    for col_name in col_names:
        print(col_name)
        df[col_name] = df[col_name].str.replace('.', ',')

        split_values = df[col_name].str.split(',').explode()
        unique_values = split_values.unique()

        for i, value in enumerate(unique_values):
            df[f"{col_name}_{i}"] = 0
            df[f"{col_name}_{i}"] = df[f"{col_name}_{i}"].astype(int)

        for index, row in df.iterrows():
            values = str(row[col_name]).split(',')
            for value in values:
                for i, unique_value in enumerate(unique_values):
                    if value.strip() == unique_value:
                        df.at[index, f"{col_name}_{i}"] = 1
                        break
        df.drop(col_name, axis=1, inplace= True)
    
    return df


In [58]:
cols = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis', 'Age_of_onsets']
cols2 = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis']
train.Treatment_of_rhinitis = train.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
train = pd.get_dummies(train, columns=cols2)
new_train = multival_onehot_enc(train, cols)

Treatment_of_athsma


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


General_cofactors


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


Treatment_of_atopic_dematitis


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


Age_of_onsets


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [130]:
# new_train

In [59]:
df_num = new_train.select_dtypes(include=object)


In [60]:
new_train = new_train.drop(df_num, axis=1)

In [74]:
cols_test = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis']
cols2_test = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis', 'Age_of_onsets']
test.Treatment_of_rhinitis = test.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
test_df = pd.get_dummies(test, columns=cols2_test)
new_test = multival_onehot_enc(test_df, cols_test)

Treatment_of_athsma
General_cofactors
Treatment_of_atopic_dematitis


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\707930443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [75]:
df_num_test = new_test.select_dtypes(include=object)


In [76]:
new_test = new_test.drop(df_num_test, axis=1)

In [77]:
new_test = new_test.drop('trustii_id', axis=1)

In [78]:
new_test.shape

(586, 401)

In [79]:
new_train.shape

(2989, 412)

In [80]:
df1 = new_train
df2 = new_test
columns_df1 = set(df1.columns)
columns_df2 = set(df2.columns)

different_columns = list(columns_df1 - columns_df2)
different_columns.extend(list(columns_df2 - columns_df1))

In [81]:
different_columns

['General_cofactors_19',
 'French_Region_regionO',
 'French_Region_regionN',
 'Age_of_onsets_7',
 'General_cofactors_16',
 'General_cofactors_20',
 'Age_of_onsets_8',
 'General_cofactors_18',
 'Treatment_of_athsma_10',
 'General_cofactors_17',
 'Treatment_of_atopic_dematitis_8']

In [89]:
df1 = new_train
df2 = new_test
common_cols = set(df1.columns).intersection(df2.columns)
X_updated = df1.reindex(columns=common_cols)
test_updated = df2.reindex(columns=common_cols)

In [92]:
print(X_updated.shape)
print(test_updated.shape)

(2989, 401)
(586, 401)


In [105]:
X_updated.to_csv("saved_csv/train_encoded.csv", index=False)
test_updated.to_csv("saved_csv/test_encoded.csv", index=False)
target.to_csv("saved_csv/raw_target.csv", index=False)

In [101]:
X_updated = X_updated.fillna(0)

In [107]:
target = target.replace(9,np.nan)

In [108]:
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [123]:
imputer = KNNImputer(n_neighbors = 4)
target_knn = pd.DataFrame(imputer.fit_transform(target), columns=target.columns)

In [127]:
def impute_threshold(df, threshold=0.55):
    return df.applymap(lambda x: 1 if x >= threshold else 0)

In [128]:
target_new = impute_threshold(target_knn)

In [130]:
target_new.to_csv("saved_csv/target_knn.csv", index=False)

In [141]:
target_temp = target.fillna(9)
target_temp.Allergy_Present.value_counts()

1.0    1368
9.0    1230
0.0     391
Name: Allergy_Present, dtype: int64

In [143]:
target_new.Allergy_Present.value_counts()

1    2598
0     391
Name: Allergy_Present, dtype: int64

In [131]:
imputer = IterativeImputer(random_state = 17)

In [132]:
target_ir = pd.DataFrame(imputer.fit_transform(target), columns=target.columns)

c:\Users\ADE17\anaconda3\anaconda2\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [144]:
target_ir.to_csv("saved_csv/target_ir.csv", index=False)

In [ ]:
target